In [152]:
import pandas as pd
from matplotlib import pyplot as plt

CSV_FILE = '../data/r_transactions.csv'
df = pd.read_csv(CSV_FILE) # ,encoding="ISO-8859-1")
df = df.drop(['地域',
              '市区町村コード',
              '都道府県名',
              '市区町村名',
              '今後の利用目的',
              '備考',
              '坪単価',
              '取引価格（�u単価）',
              '延床面積（�u）',
              '建物の用途',
              '前面道路：種類'],axis=1)

column_names_map = {
    '種類': 'type',
    '地区名': 'area',
    '最寄駅：名称': 'station',
    '最寄駅：距離（分）': 'distance_from_station',
    '取引価格（総額）': 'price',
    '間取り': 'layout',
    '面積（�u）':'floor_space',
    '土地の形状': 'land_shape',
    '間口':'frontage', 
    '建築年': 'year_built',
    '建物の構造': 'building_structure',
    '前面道路：方位': 'front_road_direction', 
    '前面道路：幅員（ｍ）':'front_road_width', 
    '都市計画':'urban_planning', 
    '建ぺい率（％）':'building_coverage',
    '容積率（％）':'floor_area_ratio',
    '取引時点': 'transaction_quarter'
}
df = df.rename(columns=column_names_map)

df = df[(df.type=='宅地(土地)') & (df.floor_space.str.contains('以上')==False)]
df = df.drop(['layout','year_built','building_structure','type'],axis=1)


In [153]:
print(df.shape)
df.isnull().sum()

(1508, 14)


No                        0
area                      0
station                   0
distance_from_station     2
price                     0
floor_space               0
land_shape                3
frontage                 98
front_road_direction      2
front_road_width         19
urban_planning            1
building_coverage         8
floor_area_ratio          8
transaction_quarter       0
dtype: int64

In [154]:
def one_hot_encode(df,cols):
    for col in cols:
        one_hot = pd.get_dummies(df[col],prefix=col)
        df = df.drop([col],axis=1)
        df = df.join(one_hot)
    return df

df = one_hot_encode(df,['area','station','land_shape','front_road_direction','urban_planning','transaction_quarter'])
print(df.shape)
    


(1508, 129)


In [162]:
df = df.fillna(method='ffill')
print(df.shape)
print(df.mean().shape)
df.frontage.mean()

(1508, 129)
(128,)


TypeError: Could not convert 10.611.51010.110.54.54.516.58.239582.77.378.94.54.818.86.4666.56.56.77.810.57.17.3239.91169355.94.510.978.5413.678.972.56.54.5774.55.59102.72.6710.13.77109.610.5108910.5145.581098.538102.51769.29.28.511.5122.539.213.564.75610322.6108.710.5810.736.599.76.19.59.98.57.56.76.511.112.57.37.32.81832.26.48.55.389611.38.55.577.2118.765.35.247.8139.25.37.316.87510.87.36318.17.39.1106.512101515136.21422012.57.314.62.563.4678.27815663.72.52.57.52.595.46.36.36.68.68.91685.99955.76.115.58.36.99.59.4715.113.516.510101515638.52.55.3514.58.31586812.515898.9811.26.310.46.255366.316.56641010137.67.52.71015.1154.712.86.816.519199.36.5612.594.35563191514.365.851685.16.56.412.52.54.44.44.47.34.4813.22.655.610.52.513.313.38.75.262.56.411.77.88.45.23.614.835.35915129.355.22.55.66.5129.85.4116.66.996.65.75.8557.12.59.84.55.5127.395.89.217.1810.96.64.2115.86.5202.56115.25.29759.520.639.89.33313.513.52.172281311117.913274.52.54.55.447.59693.576.56.56.55.49196.66.6176.72.943665.51212

In [96]:
sorted(df.columns)

['No',
 'area_三田',
 'area_上目黒',
 'area_下目黒',
 'area_中央町',
 'area_中根',
 'area_中町',
 'area_中目黒',
 'area_五本木',
 'area_八雲',
 'area_南',
 'area_原町',
 'area_大岡山',
 'area_大橋',
 'area_平町',
 'area_東が丘',
 'area_東山',
 'area_柿の木坂',
 'area_洗足',
 'area_目黒',
 'area_目黒本町',
 'area_碑文谷',
 'area_祐天寺',
 'area_緑が丘',
 'area_自由が丘',
 'area_青葉台',
 'area_駒場',
 'area_鷹番',
 'building_coverage',
 'distance_from_station',
 'floor_area_ratio',
 'floor_space',
 'front_road_direction_北',
 'front_road_direction_北東',
 'front_road_direction_北西',
 'front_road_direction_南',
 'front_road_direction_南東',
 'front_road_direction_南西',
 'front_road_direction_接面道路無',
 'front_road_direction_東',
 'front_road_direction_西',
 'front_road_width',
 'frontage',
 'land_shape_ほぼ台形',
 'land_shape_ほぼ整形',
 'land_shape_ほぼ正方形',
 'land_shape_ほぼ長方形',
 'land_shape_不整形',
 'land_shape_台形',
 'land_shape_正方形',
 'land_shape_袋地等',
 'land_shape_長方形',
 'price',
 'station_不動前',
 'station_中目黒',
 'station_代官山',
 'station_大岡山',
 'station_学芸大学',
 'station_恵比寿',


In [160]:
sorted(df.mean().index)

['No',
 'area_三田',
 'area_上目黒',
 'area_下目黒',
 'area_中央町',
 'area_中根',
 'area_中町',
 'area_中目黒',
 'area_五本木',
 'area_八雲',
 'area_南',
 'area_原町',
 'area_大岡山',
 'area_大橋',
 'area_平町',
 'area_東が丘',
 'area_東山',
 'area_柿の木坂',
 'area_洗足',
 'area_目黒',
 'area_目黒本町',
 'area_碑文谷',
 'area_祐天寺',
 'area_緑が丘',
 'area_自由が丘',
 'area_青葉台',
 'area_駒場',
 'area_鷹番',
 'building_coverage',
 'distance_from_station',
 'floor_area_ratio',
 'floor_space',
 'front_road_direction_北',
 'front_road_direction_北東',
 'front_road_direction_北西',
 'front_road_direction_南',
 'front_road_direction_南東',
 'front_road_direction_南西',
 'front_road_direction_接面道路無',
 'front_road_direction_東',
 'front_road_direction_西',
 'front_road_width',
 'land_shape_ほぼ台形',
 'land_shape_ほぼ整形',
 'land_shape_ほぼ正方形',
 'land_shape_ほぼ長方形',
 'land_shape_不整形',
 'land_shape_台形',
 'land_shape_正方形',
 'land_shape_袋地等',
 'land_shape_長方形',
 'price',
 'station_不動前',
 'station_中目黒',
 'station_代官山',
 'station_大岡山',
 'station_学芸大学',
 'station_恵比寿',
 'station_東北沢